# Create dictionaries for SUMup and SNOWPACK

#### Author: Megan Thompson-Munson
#### Date created: 20 September 2021

This script reads in SUMup observation data and SNOWPACK output, reformats the data, and creates dictionaries that are saved as pickle files.

TO DO:
- Add CFM capability
- Add ```.smet``` capability
- Add more comments about input/output

## User input

In [9]:
# BEGIN

# Select ice sheet
icesheet = 'GrIS'

# Give path of SNOWPACK output data
path = '/projects/metm9666/snowpack/Scripts/Spinup/output/'

# END

In [10]:
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime
import pickle

## Read in SUMup data

In [28]:
# Open latest SUMup dataset
sumup = xr.open_dataset('sumup_density_2020_v060121.nc')

# Extract data and remove no data
su_elev = sumup['Elevation'].values
su_lat = sumup['Latitude'].values

# Ignore any sea ice data
condition = (su_elev>0) & (su_lat>-91)

# Exctract data based on condition
su_lon = sumup['Longitude'].values[condition]
su_depth0 = sumup['Start_Depth'].values[condition]
su_depth1 = sumup['Stop_Depth'].values[condition]
su_midpoint = sumup['Midpoint'].values[condition]
su_density = sumup['Density'].values[condition]
su_citation = sumup['Citation'].values[condition]
su_date = sumup['Date'].values[condition]
su_method = sumup['Method'].values[condition]
su_elev = su_elev[condition]
su_lat = su_lat[condition]

# Create new array of midpoints greater than -9999 (note that last entry in the dataset has no depth)
a = su_midpoint
b = (su_depth0+su_depth1)/2
M = []
for i in range(len(a)):
    if a[i] < b[i]:
        m = b[i]
    if a[i] >= b[i]:
        m = a[i]
    M.append(m)

# Some dates are just year (e.g., 'YYYY0000') so this will create a new column with Jan 1 of the year as the date
su_timestamp = []
for i in range(len(su_date)):
    d = su_date[i]
    date_str = str(d)
    
    # These particular dates appear to be very incorrect
    if date_str == '19999000.0':
        date_str = '19990000.0'
    if date_str == '20089620.0':
        date_str = '20080620.0'
    
    year = date_str[0:4]
    month = date_str[4:6]
    day = date_str[6:8]
    
    # Add Jan 1 to year-only dates, and change any with 32 days to 31 days
    if month == '00':
        month = '01'
    if day == '00':
        day = '01'
    if day == '32':
        day = '31'
    
    d = float(year+month+day)
    su_timestamp.append(d)

su_timestamp = np.array(su_timestamp)
    
# Create SUMup dataframe
su_data = {'Citation':su_citation,'Timestamp':su_timestamp,'Latitude':su_lat,'Longitude':su_lon,
              'Elevation':su_elev,'Midpoint':M,'StartDepth':su_depth0,'StopDepth':su_depth1,
              'Thickness':su_depth1-su_depth0,'Density':su_density*1000}

df = pd.DataFrame(data=su_data)

# Turn date into timestamp
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y%m%d')

# Create a unique index for each core
n = -1
id0 = []
for i in range(len(su_citation)-1):
    if (su_citation[i]==su_citation[i-1] and su_lat[i]==su_lat[i-1] and su_lon[i]==su_lon[i-1] and su_method[i]==su_method[i-1]):
        index = n
    else:
        n += 1
        index = n
    id0.append(index)
id0.append(id0[-1])

# Give each datapoint within a core index its own index
m = -1
id1 = []
for i in range(len(id0)-1):
    if id0[i] == id0[i-1]:
        m += 1
    else:
        m = 0
    id1.append(m)
id1.append(id1[-1]+1)

# Set indices in dataframe
df['CoreID'] = id0
df['CoreIdx'] = id1
idx0 = pd.Series(data=id0,name='Core')
idx1 = pd.Series(data=id1,name='Index')
idx_arrays = [idx0,idx1]
df.index = idx_arrays

if icesheet == 'GrIS':
    df = df[df.Latitude>0]

df

Citation  Timestamp   Latitude  Longitude  Elevation  Midpoint  \
Core Index                                                                   
6    0          11.0 2011-05-12  73.647102  -38.67720     3108.0     0.050   
     1          11.0 2011-05-12  73.647102  -38.67720     3108.0     0.150   
     2          11.0 2011-05-12  73.647102  -38.67720     3108.0     0.250   
     3          11.0 2011-05-12  73.647102  -38.67720     3108.0     0.350   
     4          11.0 2011-05-12  73.647102  -38.67720     3108.0     0.450   
...              ...        ...        ...        ...        ...       ...   
1319 5         187.0 2013-06-04  72.579781  -38.45863     3210.0     0.765   
     6         187.0 2013-06-04  72.579781  -38.45863     3210.0     0.795   
     7         187.0 2013-06-04  72.579781  -38.45863     3210.0     0.825   
     8         187.0 2013-06-04  72.579781  -38.45863     3210.0     0.855   
     9         187.0 2013-06-04  72.579781  -38.45863     3210.0     0.885   

            StartDepth  StopDepth  Thickness     Density  CoreID  CoreIdx  
Core Index                                                                 
6    0            0.00       0.10       0.10  321.000000       6        0  
     1            0.10       0.20       0.10  351.000000       6        1  
     2            0.20       0.30       0.10  363.000000       6        2  
     3            0.30       0.40       0.10  266.000000       6        3  
     4            0.40       0.50       0.10  281.000000       6        4  
...                ...        ...        ...         ...     ...      ...  
1319 5            0.75       0.78       0.03  274.000000    1319        5  
     6            0.78       0.81       0.03  297.699982    1319        6  
     7            0.81       0.84       0.03  308.000000    1319        7  
     8            0.84       0.87       0.03  305.599976    1319        8  
     9            0.87       0.90       0.03  326.299988    1319        9  

[1652433 rows x 12 columns]

## Metadata about both datasets

In [29]:
# Extract unqiue lat/lon from SUMup and save as new dataframe
df_sumuplocs = df[df.CoreIdx==0][['CoreID','Citation','Timestamp','Latitude','Longitude']]
df_sumuplocs = df_sumuplocs.reset_index(drop=True)

# Read in MERRA-2 location data
df_merra2locs = pd.read_table('GrIS_full_station_list.lst',skiprows=1,delim_whitespace=True,usecols=[0,3,4],names=['Station','Latitude','Longitude'])

# Extract VIRs
VIRs = []
for i in range(len(df_merra2locs)):
    VIR = df_merra2locs.Station[i][8:]
    VIRs.append(VIR)
df_merra2locs['VIR'] = VIRs
df_merra2locs.drop(columns=['Station'])

# Haversine formula for calculating distance between two points on Earth
def haversine(lat1,lon1,lat2,lon2):
    phi1 = np.deg2rad(lat1)
    phi2 = np.deg2rad(lat2)
    theta1 = np.deg2rad(lon1)
    theta2 = np.deg2rad(lon2)
    del_phi = phi2-phi1
    del_theta = theta2-theta1
    a = np.sin(del_phi/2)**2 + (np.cos(phi1)*np.cos(phi2)*np.sin(del_theta/2)**2)
    c = 2*np.arctan2(np.sqrt(a),np.sqrt(1-a))
    d = (6371e3)*c # Earth's radius in meters
    return d # Meters

# Function for finding closest MERRA-2 location to given SUMup location
def closest_location(sumuplat,sumuplon):
    distance = []
    for i in range(len(df_merra2locs)):
        lat1 = sumuplat
        lon1 = sumuplon
        lat2 = df_merra2locs.Latitude[i]
        lon2 = df_merra2locs.Longitude[i]
        d = haversine(lat1,lon1,lat2,lon2)
        distance.append(d)
    p = np.where(distance == min(distance))
    return df_merra2locs.loc[p]

metadata = np.zeros((len(df_sumuplocs),7))
for i in range(len(df_sumuplocs)):
    metadata[i,0] = df_sumuplocs.CoreID[i]
    metadata[i,1] = np.array(df_sumuplocs.Timestamp)[i]
    metadata[i,2] = df_sumuplocs.Latitude[i]
    metadata[i,3] = df_sumuplocs.Longitude[i]
    merra2 = closest_location(df_sumuplocs.Latitude[i],df_sumuplocs.Longitude[i])
    metadata[i,4] = merra2.VIR.values[0]
    metadata[i,5] = merra2.Latitude.values[0]
    metadata[i,6] = merra2.Longitude.values[0]

# Create dataframe of metadata and turn float date back into timestamp
df_meta = pd.DataFrame(metadata,columns=['CoreID','Timestamp','SUMupLat','SUMupLon',
                                         'VIR','MERRALat','MERRALon'])
df_meta['Timestamp'] = pd.to_datetime(df_meta['Timestamp'])

# Ignore anny missing files
df_meta = df_meta[(df_meta.VIR!=107)]
df_meta.reset_index(drop=True,inplace=True)

In [30]:
df_meta

,CoreID,Timestamp,SUMupLat,SUMupLon,VIR,MERRALat,MERRALon
0,6.0,2011-05-12,73.647102,-38.677200,494.0,73.5,-38.750
1,7.0,2011-05-24,74.507965,-41.339031,574.0,74.5,-41.250
2,8.0,2011-05-12,75.129303,-40.541832,627.0,75.0,-40.625
3,9.0,2010-05-20,75.408035,-41.068619,678.0,75.5,-41.250
4,10.0,2010-05-20,75.949409,-42.160610,730.0,76.0,-41.875
...,...,...,...,...,...,...,...
421,1315.0,2008-06-08,72.549934,-38.309067,417.0,72.5,-38.125
422,1316.0,2008-06-16,72.579552,-38.505466,416.0,72.5,-38.750
423,1317.0,2008-06-20,72.549934,-38.309067,417.0,72.5,-38.125
424,1318.0,2013-06-04,72.635132,-38.514919,416.0,72.5,-38.750


## Read in SNOWPACK data and create dictionaries

In [45]:
dict_list = []

for i in range(len(df_meta)):
    
    # Select line of metadata
    meta = df_meta.loc[i]
    
    if icesheet == 'GrIS':
        file = path+'VIR'+str(int(meta.VIR))+'_GrIS.pro'
        
    # Open *.pro file and read in header (44 lines in length)
    f = open(file,'r')
    for j in range(44):
        header = f.readline()
        if j == 1:
            VIR = int(header[29:-1])
        if j == 2:
            latitude_sp = float(header[10:-1])
        if j == 3:
            longitude_sp = float(header[11:-1])
        if j == 4:
            elevation_sp = float(header[9:-1])
    
    timestamps_sp = [] # Empty list for storing SNOWPACK timestamps

    # Read data line by line
    data = f.readlines()
    for line in data:
        linecode = line[0:4] # SNOWPACK gives each data type a 4-digit code

        # Extract timestamps and save in a list
        if linecode == '0500':
            raw_date_sp = line[5:24]
            date_sp = datetime.strptime(raw_date_sp,'%d.%m.%Y %H:%M:%S')
            timestamp_sp = pd.to_datetime(date_sp)
            timestamps_sp.append(timestamp_sp)

    # Find SNOWPACK timestamp that's closest to the desired SUMup one
    closest = min(timestamps_sp, key=lambda sub: abs(sub - meta.Timestamp))
    k = np.where(np.array(timestamps_sp)==closest)[0][0]

    # Read data and extract lines corresponding to closest timestamp
    for line in data:
        linecode = line[0:4] # SNOWPACK gives each data type a 4-digit code

        if linecode == '0500':
            raw_date_sp = line[5:24]
            date_sp = datetime.strptime(raw_date_sp,'%d.%m.%Y %H:%M:%S')
            timestamp_sp = pd.to_datetime(date_sp)

            if timestamp_sp == closest:

                index = k*27 # Each timestamp has 27 elements, so this allows us to get to the start of each new timestamp

                # Extract variables of interest by spliting the lines and creating lists of the data
                height = list(map(float,data[index+1][5:-1].split(',')))[1:] # Height (cm) (converted to m in dataframe)
                h = np.array(height) # Create array of height for conversion to depth 
                depth = (h-h[-1])*-1 # Depth sets surface as 0
                density = list(map(float,data[index+2][5:-1].split(',')))[1:] # Density (kg/m^3)
                temperature = list(map(float,data[index+3][5:-1].split(',')))[1:] # Temperature (dec C)
                water = list(map(float,data[index+6][5:-1].split(',')))[1:] # Water content (%)
                ice = list(map(float,data[index+14][5:-1].split(',')))[1:] # Ice content (%)
                air = list(map(float,data[index+15][5:-1].split(',')))[1:] # Air content (%)
                t = timestamp_sp
                
    dict_sp = {'ID':meta.VIR,'Timestamp':t,'Elevation':elevation_sp,'Latitude':latitude_sp,'Longitude':longitude_sp,'Height':np.array(height)/100,
               'Depth':depth/100,'Density':np.array(density),'Temperature':np.array(temperature),'Ice':np.array(ice)/100,'Air':np.array(air)/100,'Water':np.array(water)/100}
    
    # Create SUMup dictionary for corresponding core
    df_core = df[df.CoreID==meta.CoreID]
    
    coreid = df_core.CoreID.values[0]
    citation_su = df_core.Citation.values[0]
    timestamp_su = df_core.Timestamp.values[0]
    latitude_su = df_core.Latitude.values[0]
    longitude_su = df_core.Longitude.values[0]
    elevation_su = df_core.Elevation.values[0]
    
    midpoint_su = np.array(df_core.Midpoint)
    startdepth_su = np.array(df_core.StartDepth)
    stopdepth_su = np.array(df_core.StopDepth)
    thickness_su = np.array(df_core.Thickness)
    density_su = np.array(df_core.Density)
    
    dict_su = {'CoreID':coreid,'Citation':citation_su,'Timestamp':timestamp_su,'Latitude':latitude_su,'Longitude':longitude_su,
               'Elevation':elevation_su,'Midpoint':midpoint_su,'StartDepth':startdepth_su,'StopDepth':stopdepth_su,'Thickness':thickness_su,'Density':density_su}
    
    dictionaries = {'SNOWPACK':dict_sp,'SUMup':dict_su}
    
    dict_list.append(dictionaries)
    
    f.close()

## Pickle the dictionaries

In [46]:
pickle.dump(dict_list, open(icesheet+'_data.p','wb'))